In [2]:
library('clusterProfiler')
library(org.Mm.eg.db)
library(ggplot2)

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:clusterProfiler’:

    rename


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname



Attaching package: ‘IRanges’


The following object is masked from ‘package:clusterProfiler’:

    slice



Attaching package: ‘AnnotationDbi’


The following object is masked from ‘package:clusterProfiler’:

    select






In [3]:
df.deg <- read.csv("/data2st1/junyi/output/atac0416/dar/region_nt/df_annotattion_mememto.csv")

In [4]:
head(df.deg)

,names,tx,de_coef,de_se,de_pval,dv_coef,dv_se,dv_pval,ctname,region,condition,gene_name,gene_id,gstart,gend,strand,annotation,distance,de_pval_adj
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<dbl>
1,chr15:75086569-75087070,stim,0.4328717,0.02169902,1.011588e-88,-0.2990962,0.12576227,1.999109e-02,PFC_Glut,PFC,MC,Gm28502,ENSMUSG00000099452.1,75076209,75079094,-,genebody,-7476,9.510063e-86
2,chr12:120028551-120029052,stim,0.4805240,0.02477918,5.125947e-84,0.2256055,0.12031550,5.658753e-02,PFC_Glut,PFC,MC,Tmem196,ENSMUSG00000048004.15,119945821,120021246,+,genebody,-7306,3.234648e-81
3,chrX:143482812-143483313,stim,0.6766919,0.04040743,3.030414e-63,-0.4103432,0.15793791,1.027334e-02,PFC_Glut,PFC,MC,Pak3,ENSMUSG00000031284.16,143518590,143797796,+,genebody,35278,4.431668e-61
4,chr2:98662045-98662546,stim,0.5046512,0.03123931,5.452844e-59,-1.2121796,0.09370949,5.131220e-38,PFC_Glut,PFC,MC,Gm10801,ENSMUSG00000075015.3,98662236,98664083,+,genebody,0,6.164123e-57
5,chr2:98662628-98663129,stim,0.5258591,0.03770465,1.715753e-44,-0.7587388,0.09776371,1.811849e-14,PFC_Glut,PFC,MC,Gm10801,ENSMUSG00000075015.3,98662236,98664083,+,genebody,0,7.114042e-43
6,chr6:103648949-103649450,stim,0.4805918,0.03522453,1.141932e-42,-0.4912399,0.14027693,4.267667e-04,PFC_Glut,PFC,MC,Chl1,ENSMUSG00000030077.11,103510585,103750211,+,genebody,0,4.188192e-41


In [5]:
# 获取所有唯一的ct_name和condition组合
ct.names <- c("AMY_Glut", "AMY_GABA", "HIP_Glut", "HIP_GABA", "PFC_Glut", "PFC_GABA")
conditions <- c("MC", "MW")

outfolder <- "/data2st1/junyi/output/atac0416/dar/region_nt/"


In [7]:
# 转换ID（Symbol → Entrez）

# ctname <- "PFC_Glut"
# condition <- "MW"

for (ctname in ct.names) {
  for (condition in conditions) {
    # 筛选数据
    tryCatch(
      {
        print(paste("Processing:", ctname, condition))

        subset_df <- df.deg[df.deg$ctname == ctname & df.deg$condition == condition, ]

        print(paste("Number of genes:", nrow(subset_df)))

        gene_entrez <- bitr(unique(subset_df$gene_name), 
                            fromType = "SYMBOL", 
                            toType = "ENTREZID", 
                            OrgDb = "org.Mm.eg.db")
                            
        entrez_ids <- gene_entrez$ENTREZID
        ego <- enrichGO(gene = entrez_ids,
                        OrgDb = org.Mm.eg.db,
                        keyType = "ENTREZID",
                        ont = "ALL",  # "BP", "MF", "CC" 或 "ALL"
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05,
                        readable = TRUE)  # 结果中显示基因符号

        # 查看结果
        write.csv(ego, file = paste(outfolder, "GO_enrichment_memento",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        p <- dotplot(ego, showCategory = 30)
        ggsave(paste(outfolder, "GO_enrichment_memento_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        p<-barplot(ego, showCategory = 30)  # 条形图
        ggsave(paste(outfolder, "GO_enrichment_memento_bar",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        kk <- enrichKEGG(gene = entrez_ids,
                        organism = "mmu",  # 小鼠KEGG代码
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05)

        # 可视化
        p<-dotplot(kk, showCategory = 30)
        write.csv(kk, file = paste(outfolder, "KEGG_enrichment_memento",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        ggsave(paste(outfolder, "kegg_enrichment_memento_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸
      }, 
      error = function(e) {
        print(paste("Error in subsetting data for", ctname, condition))
    })

  }
}

[1] "Processing: AMY_Glut MC"
[1] "Number of genes: 5354"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“8.04% of input gene IDs are fail to map...”
Reading KEGG annotation online: "https://rest.kegg.jp/link/mmu/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/mmu"...



[1] "Processing: AMY_Glut MW"
[1] "Number of genes: 9056"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“13.46% of input gene IDs are fail to map...”


[1] "Processing: AMY_GABA MC"
[1] "Number of genes: 6850"


...

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“11.89% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MC"
[1] "Number of genes: 313"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“9.75% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MW"
[1] "Number of genes: 9884"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“8.49% of input gene IDs are fail to map...”
